<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>

<h1 align=center><font size = 5>Assignment: SQL Notebook for Peer Assignment</font></h1>

Estimated time needed: **60** minutes.

## Introduction
Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions 


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [45]:
!pip install sqlalchemy==1.3.9
!pip install ipython-sql
!pip install ipython-sql prettytable
!pip install ibm_db_sa
!pip install -q pandas

  Using cached SQLAlchemy-1.3.9-cp311-cp311-linux_x86_64.whl
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.3.9 which is incompatible.
jupyterhub 5.2.1 requires SQLAlchemy>=1.4.1, but you have sqlalchemy 1.3.9 which is incompatible.
  Using cached SQLAlchemy-2.0.36-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.7 kB)
Using cached SQLAlchemy-2.0.36-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.2 MB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.9
    Uninstalling SQLAlchemy-1.3.9:
      Successfully uninstalled SQLAlchemy-1.3.9


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [46]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [47]:
%sql ibm_db_sa://css12979:onDiZbYqVnKzqYb6@815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:30367/bludb?security=SSL

**Note:This below code is added to allow us to use the deprecated magic line %sql syntax**


In [48]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [49]:
# Number of columns
%sql SELECT colcount AS "Number of Columns" FROM syscat.tables WHERE tabname='SPACEX';

 * ibm_db_sa://css12979:***@815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:30367/bludb?security=SSL
Done.


Number of Columns
10


In [50]:
# Number of Rows
%sql SELECT count(*) AS "Number of Rows" FROM SPACEX;

 * ibm_db_sa://css12979:***@815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:30367/bludb?security=SSL
Done.


Number of Rows
101


In [51]:
# Column names and info
%sql SELECT colno, colname, typename, length FROM syscat.columns WHERE tabname='SPACEX' ORDER BY colno;

 * ibm_db_sa://css12979:***@815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:30367/bludb?security=SSL
Done.


colno,colname,typename,length
0,DATE,DATE,4
1,TIME__UTC_,VARCHAR,8
2,BOOSTER_VERSION,VARCHAR,14
3,LAUNCH_SITE,VARCHAR,12
4,PAYLOAD,VARCHAR,61
5,PAYLOAD_MASS__KG_,INTEGER,4
6,ORBIT,VARCHAR,11
7,CUSTOMER,VARCHAR,57
8,MISSION_OUTCOME,VARCHAR,32
9,LANDING_OUTCOME,VARCHAR,22


In [52]:
# First 5 rows
%sql SELECT * FROM SPACEX LIMIT 5;

 * ibm_db_sa://css12979:***@815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:30367/bludb?security=SSL
Done.


DATE,time__utc_,booster_version,launch_site,payload,payload_mass__kg_,orbit,customer,mission_outcome,landing_outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


## Tasks

Now write and execute SQL queries to solve the assignment tasks.

**Note: If the column names are in mixed case enclose it in double quotes
   For Example "Landing_Outcome"**

### Task 1




##### Display the names of the unique launch sites  in the space mission


In [53]:
# Task 1 ANSWER
%sql SELECT DISTINCT LAUNCH_SITE FROM SPACEX;

 * ibm_db_sa://css12979:***@815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:30367/bludb?security=SSL
Done.


launch_site
CCAFS LC-40
CCAFS SLC-40
KSC LC-39A
VAFB SLC-4E



### Task 2


#####  Display 5 records where launch sites begin with the string 'CCA' 


In [54]:
# Task 2 ANSWER
%sql SELECT * FROM SPACEX WHERE launch_site LIKE 'CCA%' LIMIT 5;

 * ibm_db_sa://css12979:***@815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:30367/bludb?security=SSL
Done.


DATE,time__utc_,booster_version,launch_site,payload,payload_mass__kg_,orbit,customer,mission_outcome,landing_outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


### Task 3




##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [55]:
# Task 3 ANSWER
%sql SELECT sum(payload_mass__kg_) AS "Total Payload Mass (kg)" FROM SPACEX WHERE customer LIKE '%NASA (CRS)%';

 * ibm_db_sa://css12979:***@815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:30367/bludb?security=SSL
Done.


Total Payload Mass (kg)
48213


### Task 4




##### Display average payload mass carried by booster version F9 v1.1


In [56]:
# Task 4 ANSWER
%sql SELECT sum(payload_mass__kg_) / count(payload_mass__kg_) AS "Average Payload Mass (kg)" FROM SPACEX WHERE booster_version LIKE 'F9 v1.1';

 * ibm_db_sa://css12979:***@815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:30367/bludb?security=SSL
Done.


Average Payload Mass (kg)
2928


### Task 5

##### List the date when the first succesful landing outcome in ground pad was acheived.


_Hint:Use min function_ 


In [57]:
# Task 5 ANSWER
%sql SELECT min(DATE) AS "First Successful Landing Outcome Date" FROM SPACEX WHERE landing_outcome LIKE 'Success (ground pad)';

 * ibm_db_sa://css12979:***@815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:30367/bludb?security=SSL
Done.


First Successful Landing Outcome Date
2015-12-22


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [58]:
# Task 6 ANSWER
%sql SELECT DISTINCT booster_version FROM SPACEX WHERE landing_outcome = 'Success (drone ship)' and payload_mass__kg_ BETWEEN 4000 and 6000;

 * ibm_db_sa://css12979:***@815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:30367/bludb?security=SSL
Done.


booster_version
F9 FT B1021.2
F9 FT B1031.2
F9 FT B1022
F9 FT B1026


### Task 7




##### List the total number of successful and failure mission outcomes


In [59]:
# Successful mission outcomes
%sql SELECT count(*) AS "Success" FROM SPACEX WHERE lcase(landing_outcome) LIKE '%success%';


 * ibm_db_sa://css12979:***@815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:30367/bludb?security=SSL
Done.


Success
61


In [60]:
# Failure mission outcomes
%sql SELECT count(*) AS "Failure" FROM SPACEX WHERE lcase(landing_outcome) NOT LIKE '%success%';

 * ibm_db_sa://css12979:***@815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:30367/bludb?security=SSL
Done.


Failure
40


In [61]:
# Task 7 ANSWER
%sql SELECT (SELECT count(*) FROM SPACEX WHERE lcase(landing_outcome) LIKE '%success%') AS "Success", count(*) AS "Failure" FROM SPACEX WHERE lcase(landing_outcome) NOT LIKE '%success%';

 * ibm_db_sa://css12979:***@815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:30367/bludb?security=SSL
Done.


Success,Failure
61,40


### Task 8



##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [62]:
# Maximum Payload Mass (kg)
%sql SELECT max(payload_mass__kg_) AS "Max Payload Mass (kg)" FROM SPACEX;

 * ibm_db_sa://css12979:***@815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:30367/bludb?security=SSL
Done.


Max Payload Mass (kg)
15600


In [63]:
# Task 8 ANSWER
%sql SELECT booster_version, payload_mass__kg_ FROM SPACEX WHERE payload_mass__kg_ = (SELECT max(payload_mass__kg_) FROM SPACEX);

 * ibm_db_sa://css12979:***@815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:30367/bludb?security=SSL
Done.


booster_version,payload_mass__kg_
F9 B5 B1048.4,15600
F9 B5 B1049.4,15600
F9 B5 B1051.3,15600
F9 B5 B1056.4,15600
F9 B5 B1048.5,15600
F9 B5 B1051.4,15600
F9 B5 B1049.5,15600
F9 B5 B1060.2,15600
F9 B5 B1058.3,15600
F9 B5 B1051.6,15600


### Task 9


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.**


In [64]:
# Number of Unique Launch Dates
%sql SELECT count(DISTINCT Date) AS "Number of Unique Launch Dates" FROM SPACEX;

 * ibm_db_sa://css12979:***@815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:30367/bludb?security=SSL
Done.


Number of Unique Launch Dates
101


In [65]:
# Selecting Year, Month, Day, and Day of week from 'Date' information.
%sql SELECT DATE, YEAR(DATE) AS "Year", MONTH(DATE) AS "Month", DAY(DATE) AS "Day", DAYOFWEEK(DATE) AS "day_of_week" FROM SPACEX LIMIT 5;

 * ibm_db_sa://css12979:***@815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:30367/bludb?security=SSL
Done.


DATE,Year,Month,Day,day_of_week
2010-06-04,2010,6,4,6
2010-12-08,2010,12,8,4
2012-05-22,2012,5,22,3
2012-10-08,2012,10,8,2
2013-03-01,2013,3,1,6


### Task 10




##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [66]:
# Selecting Year, Month, Day, and Day of week from 'Date' information.
%sql SELECT DATE, YEAR(DATE) AS "Year", MONTHNAME(DATE) AS "Month", DAY(DATE) AS "Day", DAYOFWEEK(DATE) AS "day_of_week", DAYNAME(DATE) AS "day_of_week" FROM SPACEX LIMIT 5;

 * ibm_db_sa://css12979:***@815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:30367/bludb?security=SSL
Done.


DATE,Year,Month,Day,day_of_week,day_of_week_1
2010-06-04,2010,June,4,6,Friday
2010-12-08,2010,December,8,4,Wednesday
2012-05-22,2012,May,22,3,Tuesday
2012-10-08,2012,October,8,2,Monday
2013-03-01,2013,March,1,6,Friday


In [67]:
# Rank the counts (of ALL landing_outcomes between the date 04-06-2010 and 20-03-2017) in descending order.
%sql SELECT landing_outcome, count(landing_outcome) AS "Count" FROM SPACEX WHERE DATE BETWEEN '2010-06-04' AND '2017-03-20' GROUP BY landing_outcome ORDER BY count(landing_outcome) DESC;

 * ibm_db_sa://css12979:***@815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:30367/bludb?security=SSL
Done.


landing_outcome,Count
No attempt,10
Failure (drone ship),5
Success (drone ship),5
Controlled (ocean),3
Success (ground pad),3
Failure (parachute),2
Uncontrolled (ocean),2
Precluded (drone ship),1


### Reference Links

* <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : String Patterns, Sorting and Grouping</a>  

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?origin=www.coursera.org">Hands-on Lab: Built-in functions</a>

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*  <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb">Hands-on Lab: Analyzing a real World Data Set</a>




## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


<!--
## Change log
| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2024-07-10 | 1.1 |Anita Verma | Changed Version|
| 2021-07-09 | 0.2 |Lakshmi Holla | Changes made in magic sql|
| 2021-05-20 | 0.1 |Lakshmi Holla | Created Initial Version |
-->


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
